In [1]:
!pip install --quiet transformers


In [2]:
# Pull down the raw CSVs from GitHub
!wget -q https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
!wget -q https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv

In [3]:
import pandas as pd

train_df = pd.read_csv(
    "train.csv",
    header=None,
    names=["label","title","description"]
)
test_df = pd.read_csv(
    "test.csv",
    header=None,
    names=["label","title","description"]
)

train_df["text"] = train_df["title"] + " " + train_df["description"]
test_df["text"]  = test_df["title"]  + " " + test_df["description"]

In [6]:

print(train_df.shape, test_df.shape)
print(train_df.head())

(120000, 4) (7600, 4)
   label                                              title  \
0      3  Wall St. Bears Claw Back Into the Black (Reuters)   
1      3  Carlyle Looks Toward Commercial Aerospace (Reu...   
2      3    Oil and Economy Cloud Stocks' Outlook (Reuters)   
3      3  Iraq Halts Oil Exports from Main Southern Pipe...   
4      3  Oil prices soar to all-time record, posing new...   

                                         description  \
0  Reuters - Short-sellers, Wall Street's dwindli...   
1  Reuters - Private investment firm Carlyle Grou...   
2  Reuters - Soaring crude prices plus worries\ab...   
3  Reuters - Authorities have halted oil export\f...   
4  AFP - Tearaway world oil prices, toppling reco...   

                                                text  
0  Wall St. Bears Claw Back Into the Black (Reute...  
1  Carlyle Looks Toward Commercial Aerospace (Reu...  
2  Oil and Economy Cloud Stocks' Outlook (Reuters...  
3  Iraq Halts Oil Exports from Main Southe

In [7]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
!pip install --quiet nltk

In [8]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [12]:
import re
import string
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))


def preprocess_text_simple(text):
    # 1. Lowercase
    text = text.lower()
    # 2. Remove digits
    text = re.sub(r"\d+", "", text)
    # 3. Strip punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # 4. Split on whitespace
    tokens = text.split()
    # 5. Remove stopwords
    tokens = [t for t in tokens if t not in stop_words]
    # 6. Re-join
    return " ".join(tokens)

# Testing
import pandas as pd
sample = train_df.iloc[:5].copy()
sample["cleaned"] = sample["text"].apply(preprocess_text_simple)
print(sample[["text","cleaned"]])

                                                text  \
0  Wall St. Bears Claw Back Into the Black (Reute...   
1  Carlyle Looks Toward Commercial Aerospace (Reu...   
2  Oil and Economy Cloud Stocks' Outlook (Reuters...   
3  Iraq Halts Oil Exports from Main Southern Pipe...   
4  Oil prices soar to all-time record, posing new...   

                                             cleaned  
0  wall st bears claw back black reuters reuters ...  
1  carlyle looks toward commercial aerospace reut...  
2  oil economy cloud stocks outlook reuters reute...  
3  iraq halts oil exports main southern pipeline ...  
4  oil prices soar alltime record posing new mena...  


In [14]:
sample_rand = train_df.sample(n=5, random_state=42).copy()
sample_rand

,label,title,description,text
71787,3,"BBC set for major shake-up, claims newspaper","London - The British Broadcasting Corporation,...","BBC set for major shake-up, claims newspaper L..."
67218,3,Marsh averts cash crunch,Embattled insurance broker #39;s banks agree t...,Marsh averts cash crunch Embattled insurance b...
54066,2,"Jeter, Yankees Look to Take Control (AP)",AP - Derek Jeter turned a season that started ...,"Jeter, Yankees Look to Take Control (AP) AP - ..."
7168,4,Flying the Sun to Safety,When the Genesis capsule comes back to Earth w...,Flying the Sun to Safety When the Genesis caps...
29618,3,Stocks Seen Flat as Nortel and Oil Weigh,NEW YORK (Reuters) - U.S. stocks were set to ...,Stocks Seen Flat as Nortel and Oil Weigh NEW ...
